# Deploying a auto start/stop Amazon SageMaker Foundation Model endpoint backed by a API Gateway/Lambda

In this notebook, we will run through examples of managing your SageMaker endpoint with the endpoint manager functionality.

We will also walkthrough an example on how to interact with the API gateway endpoint secured by a Lambda authoerizer.

In [ ]:
import requests
import json
import time

Set your API gateway URL and auth token

`https://<API_GATEWAY_ID>.execute-api.<REGION>.amazonaws.com/prod/`

`'Authorization': '<YOUR_TOKEN>'`

In [ ]:
url='https://<API_GATEWAY_ID>.execute-api.us-east-1.amazonaws.com/prod/'
headers = {
    'Content-Type': 'application/json',
    'Accept': 'application/json',
    'Authorization': '<YOUR_TOKEN>',
    # 'X-Amzn-SageMaker-Custom-Attributes': 'accept_eula=true' # Uncomment this line if you are using a llama2
}

### Real-time Endpoint Management Functions
When you create your endpoint for the first time, it will initialize it with the default provision time in minutes. You can check the available time left on your endpoint by either querying a specific endpoint or get a list of endpoint.

#### Querying time left for a specific endpoint

You can check the time left for a specific endpoint by querying the `endpoint-expiry` api as part of the endpoint manager functionality and passing in the `EndpointName`. Below is an example on how to do this:

In [ ]:
# Set endpoint you like to lookup
endpoint_name = 'demo-Falcon40B-Endpoint'

# Flan example
# endpoint_name = 'demo-FlanT5-Endpoint'

# endpoint_name = 'demo-LLama-Endpoint'

# endpoint_name = 'demo-LLama2-Endpoint'

In [ ]:
response = requests.get(url=f"{url}/endpoint-expiry?EndpointName={endpoint_name}", headers=headers)
print(json.dumps(response.json(), indent=2))

#### Querying time left for all managed endpoints

You can also get a list of managed endpoints and their respective time left. This can be done by using the `endpoint-expiry` api as well.

In [ ]:
response = requests.get(url=f"{url}/endpoint-expiry", headers=headers)
print(json.dumps(response.json(), indent=2))

#### Extending your real-time endpoint expiry time
The managed endpoint API also provides you with the ability to extend the expiry date. Below is an example on how you can extend an endpoint by 30 minutes.

In [ ]:
payload = {
    "EndpointName": endpoint_name,
    "minutes": 30
}
response = requests.post(url=f"{url}/endpoint-expiry", headers=headers, json=payload)
print(json.dumps(response.json(), indent=2))

#### Adding a new real-time endpoint
With the endpoint management API, you can also add a new real-time endpoint with pre-existing Amazon SageMaker endpoint configurations.

Note: You can use the endpoint manager for any model regardless if it is jumpstart or not as long as you have a define Amazon SageMaker endpoint configuration.

In [ ]:
new_endpoint_name=""
new_endpoint_config_name=""

In [ ]:
payload = {
    "EndpointName": new_endpoint_name,
    "EndpointConfigName": new_endpoint_config_name,
    "minutes": 30
}
response = requests.post(url=f"{url}/endpoint-expiry", headers=headers, json=payload)
print(json.dumps(response.json(), indent=2))

### Interacting with your endpoint via API gateway

With the deploy API Gateway and model lambda, you can interact with your Amazon SageMaker endpoint through the internet via API Gateway. Below is an example on how to send your payload request to the falcon model.

In [ ]:
# Falcon example
payload = {
    "inputs": "Write a program to compute factorial in python:",
    "parameters": {"max_new_tokens": 200}
}

# FLAN example
# payload = {
#     'text_inputs':'Write a program to compute factorial in python:', 
#     'max_length': 100, 
#     'temperature': 0.0, 
#     'seed': 321
# }

# open-llama example
# payload = {
#     "text_inputs": "Building a website can be done in 10 simple steps:",
#     "max_length": 110,
#     "no_repeat_ngram_size": 3,
# }

# llama-2 example
# payload = {
#         "inputs": "Building a website can be done in 10 simple steps:",
#         "parameters" :
#             {
#                 "max_new_tokens": 110, 
#                 "top_p": 0.9, 
#                 "temperature" : 0.1
#             }
# }

response = requests.post(url=f"{url}/falcon", headers=headers, json=payload)
print(json.dumps(response.json(), indent=2))

## Asynchronously interacting with your real-time endpoint via API Gateway

In some instances, the real-time endpoint may take longer than 30 seconds to return a responses. Due to API gateway's 30 second timeout hard limit, an asynchronous approach will be required. The following code show you an example on how to asynchronously interactive with your real-time endpoint via two API calls.

In [ ]:
# Falcon example
payload = {
    "endpointname": endpoint_name,
    "body": {"inputs": "Write a program to compute factorial in python:", "parameters": {"max_new_tokens": 200}}
}

response = requests.post(url=f"{url}/startexecution", headers=headers, json=payload)
print(json.dumps(response.json(), indent=2))

# Get the execution arn
executionArn = response.json().get('executionArn')

In [ ]:
payload ={
    "executionArn": executionArn
}
response = requests.post(url=f"{url}/describeexecution", headers=headers, json=payload)

while 'RUNNING' in response.json().get('status'):
    time.sleep(1)
    response = requests.post(url=f"{url}/describeexecution", headers=headers, json=payload)
    print("Step is still running...")

# Return results
output = response.json().get('output')
output_json = json.loads(output)
body = json.loads(output_json['Body'])
body[0]['generated_text']

#### Interactive with your endpoint via Langchain/APIGateway

The deploy API can be used with the [Amazon API Gateway/Langchain](https://python.langchain.com/docs/ecosystem/integrations/amazon_api_gateway) integration.

The following example will walk you through on how to interact with the API Gateway backed by a Lambda authorizer.

In [ ]:
!pip install langchain==0.0.238

In [ ]:
from langchain.llms import AmazonAPIGateway
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType

In [ ]:
llm = AmazonAPIGateway(api_url=f"{url}/falcon", headers=headers)

### Langchain LLM example

In [ ]:
parameters = {
    "max_new_tokens": 100,
    "num_return_sequences": 1,
    "top_k": 50,
    "top_p": 0.95,
    "do_sample": False,
    "return_full_text": True,
    "temperature": 0.2,
}

prompt = "what day comes after Friday?"
llm.model_kwargs = parameters
llm(prompt)

### Langchain/APIGateway Agent Example

In [ ]:
parameters = {
    "max_new_tokens": 50,
    "num_return_sequences": 1,
    "top_k": 250,
    "top_p": 0.25,
    "do_sample": False,
    "temperature": 0.1,
}

llm.model_kwargs = parameters

# Next, let's load some tools to use. Note that the `llm-math` tool uses an LLM, so we need to pass that in.
tools = load_tools(["python_repl", "llm-math"], llm=llm)

# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

# Now let's test it out!
agent.run("""
Write a Python script that prints "Hello, world!"
""")